In [1]:
%run ./work/lib/Config.ipynb

In [2]:
# Function definitions for the rest of the workbook
import chart_studio.plotly as py
from plotly.graph_objects import *
import plotly.tools as tls
    
def columns():
    return(['food_code', 'food_desc', 'food_group_code', 'food_group_desc',
    'is_animal', 'is_dairy', 'is_red_meat', 'is_beef', 'is_other_red_meat',
    'is_white_meat', 'is_pork', 'is_poultry', 'is_sausage_or_organ_meat',
    'is_seafood', 'is_eggs', 'is_legume', 'is_nut_or_seed', 'is_bread',
    'is_other_grain_product', 'is_fruit', 'is_vegetable', 'is_white_potato',
    'is_other_vegetable', 'is_fat', 'is_sweetener',
    'pct_water', 'enerc_kcal', 'fat', 'f18d2', 'procnt', 'chocdf', 'fibtg', 'pct_fibtg', 'pct_ca', 'pct_fe', 'pct_mg', 'pct_p',
    'pct_k', 'pct_na', 'pct_zn', 'pct_cu', 'pct_mn', 'pct_se', 'pct_vitc', 'pct_thia', 'pct_ribf',
    'pct_nia', 'pct_vitb6a', 'pct_fol', 'pct_choln', 'pct_vitb12', 'pct_vita_rae', 'pct_tocpha',
    'pct_vitd', 'pct_vitk1', 'pct_f18d2', 'pct_f18d3','glycemic_index', 'cost'])

def query():
    return("SELECT * FROM contrib.food_dri_view;")

def upper_limit_query():
    return("""
        SELECT
            tagname, (tolerable_upper.amount / recommended.amount) AS pct_tolerable_upper
        FROM (
                SELECT age_from, sex, tagname, amount
                FROM dietary_reference_intake.dietary_reference_intake
                WHERE type = 'tolerable_upper'
                AND age_from = 31
                AND sex = 'Male'
        ) AS tolerable_upper
        JOIN dietary_reference_intake.dietary_reference_intake AS recommended USING(age_from, sex, tagname)
        WHERE
            recommended.type = 'recommended'
            AND (tolerable_upper.amount / recommended.amount) > 1
    """)

def get_macros(data, result):
    macros = [
        np.dot(result.x, data.loc[:, 'enerc_kcal'].to_numpy()),
        np.multiply(np.dot(result.x, data.loc[:, 'chocdf'].to_numpy()), 4),
        np.multiply(np.dot(result.x, data.loc[:, 'procnt'].to_numpy()), 4),
        np.multiply(np.dot(result.x, data.loc[:, 'fat'].to_numpy()), 9),
        np.multiply(np.dot(result.x, data.loc[:, 'f18d2'].to_numpy()), 9),
        np.dot(result.x, data.loc[:, 'fibtg'].to_numpy()),
    ]
    macros = pd.DataFrame(macros)
    macros.index = ['Calories', 'Carb Cal', 'Protein Cal', 'Fat Cal', 'Omega-6 Cal', 'Fiber gm']
    macros.columns = ['Value']
    return(macros)

def get_micros(data, result):
    # Get the nutrients for the foods
    nutrients = data.loc[:, 'pct_fibtg':'pct_f18d3'].copy()
    nutrients = (nutrients.T * result.x).T
    nutrients = nutrients[result.x>0]
    nutrients = round(nutrients, 3)
    nutrients.reset_index(drop=True, inplace=True)
    
    # Total them
    return(np.multiply(nutrients.sum(axis=0), 100))

def get_rations(data, result):
    # Get the food list
    rations = data.loc[:, 'food_code':'food_desc'][result.x>0]
    rations.reset_index(drop=True, inplace=True)
    
    # Get the amounts from the solution, converted to grams
    amounts = pd.DataFrame(np.multiply(result.x[result.x>0], 100))
    amounts.reset_index(drop=True, inplace=True)
    
    # Add the nutrients for the foods
    nutrients = data.loc[:, 'enerc_kcal':'pct_f18d3'].copy()
    nutrients = (nutrients.T * result.x).T
    nutrients = nutrients[result.x>0]
    nutrients = round(nutrients, 3)
    nutrients.reset_index(drop=True, inplace=True)

    # Combine them and label them
    rations = pd.concat((rations, amounts, nutrients), axis=1)
    cols = ['Code', 'Food', 'Amount (gm)']
    cols.extend(columns()[26:57])
    rations.columns = cols
    
    # Sort by descending amounts
    rations = rations.sort_values('Amount (gm)', ascending=False)
    rations.reset_index(drop=True, inplace=True)
    return(rations)

def summarize_solution(data, solution):
    if solution.success:
        print(get_macros(data, solution))
        print()
        print(get_micros(data, solution))
        print()
        print(get_rations(data, solution))
    return

In [3]:
# Functions for adding constraints to the nutrition LP
def add_range(data, constraints, nutrient='enerc_kcal', min=None, max=None):
    if nutrient not in data.columns:
        return(constraints)
    
    coefs = data.loc[:, nutrient].append(pd.Series(0, index=['bounds']), ignore_index=False).fillna(0)
    coefs.index = constraints.index
    
    # Validate the coefficients
    if coefs.isna().any() or np.isinf(coefs).any() or None in coefs:
        print('Bad values for nutrient ' + nutrient)
        print(coefs)
        return(constraints)
        
    if min is not None:
        coefs['bounds'] = -1*min
        constraints['min_' + nutrient] = coefs.copy()
    
    if max is not None:
        coefs['bounds'] = max
        constraints['max_' + nutrient] = coefs
    
    return(constraints)

# Functions for adding percentage of calories
def add_energy_percent_range(data, constraints=[], bounds=[], nutrient='chocdf',  mult=4, min=None, max=None):
    nut_coefs = np.multiply( np.transpose([data.loc[:, nutrient].to_numpy()]), mult )
    cal_coefs = np.transpose([data.loc[:, 'enerc_kcal'].to_numpy()])
    
    if min is not None:
        coefs = np.subtract(np.multiply(cal_coefs, min), nut_coefs)
        constraints = np.c_[constraints, coefs]
        bounds.append(0)
    
    if max is not None:
        coefs = np.subtract(nut_coefs, np.multiply(cal_coefs, max))
        constraints = np.c_[constraints, coefs]
        bounds.append(0)
    
    return(constraints, bounds)

In [5]:
# Lookup food data from the DB
data = fetch_dataframe(query())
data.columns = columns()

# Strip out any records with no nutritional value at all
data = data[np.linalg.norm(data.loc[:, 'pct_fibtg':'pct_f18d3'], axis=1) != 0]
data.reset_index(drop=True, inplace=True)

# Look up the upper limits for nutrients, if known
upper_limits = fetch_dataframe(upper_limit_query())
upper_limits.columns = ['tagname', 'amount']
for i in range(len(upper_limits)):
    upper_limits.loc[i, 'tagname'] = 'pct_' + upper_limits.loc[i, 'tagname']

print(data.shape)

(716, 59)


In [6]:
# Perform a Simplex optimization
from scipy.optimize import linprog

# Pick an objective function here:
# objective = [1 for row in constraints] # Minimize weight
# objective = np.multiply(data.loc[:, 'fibtg'].to_numpy(), -1) # Maximize fiber
# objective = data.loc[:, 'enerc_kcal'].to_numpy() # Minimize calories
objective = data.loc[:, 'chocdf'].to_numpy() # Minimize carbs
# objective = data.loc[:, 'fat'].to_numpy() # Minimize fat
# objective = np.multiply(data.loc[:, 'f18d2'].to_numpy(), -1) # Maximize Omega-6
# objective = data.loc[:, 'cost'].to_numpy() # Minimize cost
# objective = np.multiply(0.01, np.multiply(data.loc[:, 'glycemic_index'], data.loc[:, 'chocdf'])).to_numpy() # Min glycemic load

# Require 100% of every nutrient with an RDA
constraints = data.loc[:, 'pct_ca':'pct_f18d3'].fillna(0)
constraints = constraints * -1 # (-1 + 0.1)

# Label the constraints while we're at it
#constraints.columns = data.loc[:, 'pct_fibtg':'pct_f18d3'].columns
constraints.columns = data.loc[:, 'pct_ca':'pct_f18d3'].columns
constraints.index = data['food_desc']

# Add bounds to our constraints, as a bottom row
constraints = constraints.append(pd.Series(-1, index=constraints.columns, name='bounds'), ignore_index=False)

# Set calories between 1800 and 2100
# constraints = add_range(data, constraints, 'enerc_kcal', min=1800, max=2000)

# Restrict nutrients that have upper limits
for i in range(len(upper_limits)):
    tag, amount = upper_limits.loc[i]
    continue
    constraints = add_range(data, constraints, tag, min=None, max=amount)

# Restrict the remaining nutrients, because enough is as good as a feast
for tag in data.columns:
    if tag[:4] != 'pct_':
        continue
    if tag in upper_limits.loc[:, 'tagname'].to_numpy():
        continue
    constraints = add_range(data, constraints, tag, min=None, max=4)

# Limit total weight to 2.5 kilos
#constraints = np.c_[constraints, np.transpose(np.ones(len(constraints)))]
#bounds.append(25)

# Add extra fiber to our diet
#constraints = add_range(data, constraints, 'fibtg', min=5, max=None)

# Set protein between 10 and 35 percent of energy
#constraints = add_energy_percent_range(data, constraints, 'chocdf', mult=4, min=.40,     max=.65)
#constraints = add_energy_percent_range(data, constraints, 'procnt', mult=4, min=.099999, max=.35)
#constraints = add_energy_percent_range(data, constraints, 'fat',    mult=9, min=.20,     max=.35)
#constraints = add_energy_percent_range(data, constraints, 'f18d2',  mult=9, min=.01,     max=.10)

# Disallow more than a pound of any one food
limits = [(0, 4.5) for i in range(len(objective))]

# Exclude weird foods as they come up
for food_code in [63115010, 63115130, 91301030, 91301510, 91301080, 91304020, 64401000, 91304060, 55100010, 71205020, 75511010, 26205190, 75105500, 75124000, 81302050, 31110010, 31101010, 31108010, 26213100, 26123100, 26131100, 26118000, 26133100, 75236500, 75502500, 26311180, 26315180, 26315100]:
    index = np.where(data.loc[:, 'food_code'] == food_code)
    if not index[0]:
        continue
    limits[index[0][0]] = (0,0)

# Try solving that
result = linprog(objective, A_ub=constraints[:-1].to_numpy().T, bounds=limits, b_ub=constraints.loc['bounds', :].to_numpy(), options={"disp": True})
print()

# Print out the results
summarize_solution(data, result)

Running HiGHS 1.2.2 [date: 2022-08-30, git hash: n/a]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
34 rows, 688 cols, 20082 nonzeros
26 rows, 652 cols, 14525 nonzeros
Presolve : Reductions: rows 26(-8); columns 652(-64); elements 14525(-6348)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 24(446) 0s
         35     3.2388720045e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 35
Objective value     :  3.2388720045e+00
HiGHS run time      :          0.03

                   Value
Calories     3332.356742
Carb Cal       12.955488
Protein Cal  1920.104129
Fat Cal      1381.088613
Omega-6 Cal   155.428951
Fiber gm        1.017334

pct_fibtg         2.7
pct_ca          111.9
pct_fe          211.9
pct_mg          122.2
pct_p           587.0
pct_k           100.0
pct_na          523

In [7]:
rations = get_rations(data, result)
# rations.to_csv('/mnt/rations.csv')
print(rations)

        Code                                               Food  Amount (gm)  \
0   24122141  Chicken breast, baked or broiled, skin not eat...   450.000000   
1   24122171         Chicken breast, rotisserie, skin not eaten   450.000000   
2   26319180                                     Shrimp, canned   368.292529   
3   24120120  Chicken breast, NS as to cooking method, skin ...   263.511619   
4   24198500                                       Chicken feet   236.981164   
5   72130100                                    Watercress, raw   174.112260   
6   26147190                                   Sturgeon, smoked    90.223834   
7   82104000                                          Olive oil    40.253977   
8   81204000                             Ghee, clarified butter    28.510006   
9   42113000                                          Pine nuts     3.966832   
10  82103500                                       Flaxseed oil     0.347151   

    enerc_kcal     fat  f18d2   procnt 

In [8]:
-data.loc[:, 'pct_ca':'pct_f18d3'].fillna(0).T

,0,1,2,3,4,5,6,7,8,9,...,706,707,708,709,710,711,712,713,714,715
pct_ca,-0.204000,-0.143000,-0.134000,-0.290000,-0.284000,-0.200000,-0.100000,-0.100000,-0.200000,-0.200000,...,-0.241000,-0.024000,-0.055000,-0.031000,-0.005000,-0.115000,-0.037000,-0.029000,-0.011000,-0.093000
pct_fe,-0.005000,-0.007500,-0.006250,-0.036250,-0.023750,-0.000000,-0.057500,-0.057500,-0.000000,-0.000000,...,-1.155000,-0.170000,-0.101250,-0.225000,-0.256250,-0.005000,-0.018750,-0.077500,-0.012500,-0.380000
pct_mg,-0.026190,-0.030952,-0.033333,-0.064286,-0.061905,-0.016667,-0.023810,-0.059524,-0.016667,-0.016667,...,-0.690476,-0.047619,-0.064286,-0.050000,-0.047619,-0.026190,-0.028571,-0.047619,-0.016667,-0.045238
pct_p,-0.144286,-0.117143,-0.158571,-0.278571,-0.361429,-0.088571,-0.127143,-0.127143,-0.088571,-0.088571,...,-0.962857,-0.274286,-0.088571,-0.234286,-0.345714,-0.195714,-0.028571,-0.031429,-0.015714,-0.151429
pct_k,-0.035319,-0.038298,-0.043404,-0.070638,-0.078936,-0.022979,-0.033191,-0.049787,-0.022979,-0.022979,...,-0.507234,-0.077234,-0.091702,-0.044043,-0.137660,-0.030000,-0.035319,-0.034468,-0.045532,-0.024468
pct_na,-0.034667,-0.070000,-0.033333,-0.076667,-0.084667,-0.030000,-0.042000,-0.042000,-0.030000,-0.030000,...,-0.013333,-1.170667,-0.102667,-0.480667,-0.568667,-0.022667,-0.001333,-0.000667,-0.006000,-0.253333
pct_zn,-0.036364,-0.021818,-0.027273,-0.081818,-0.085455,-0.017273,-0.025455,-0.025455,-0.017273,-0.017273,...,-0.223636,-0.242727,-0.036364,-0.210000,-0.290909,-0.054545,-0.006364,-0.048182,-0.008182,-0.071818
pct_cu,-0.012222,-0.008889,-0.051111,-0.017778,-0.016667,-0.006667,-0.010000,-0.060000,-0.006667,-0.006667,...,-4.516667,-0.106667,-0.206667,-0.165556,-0.095556,-0.023333,-0.046667,-0.183333,-0.023333,-0.180000
pct_mn,-0.000000,-0.000000,-0.007826,-0.002609,-0.002609,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,-1.312174,-0.412174,-0.209130,-0.096087,-0.006522,-0.003043,-0.016957,-0.280870,-0.041304,-0.217391
pct_se,-0.038182,-0.041818,-0.025455,-0.045455,-0.269091,-0.023636,-0.034545,-0.034545,-0.023636,-0.023636,...,-0.030909,-0.610909,-0.007273,-0.378182,-0.267273,-0.225455,-0.001818,-0.007273,-0.005455,-0.547273
